# King Country House Predictions

## Background
Located in the center of Washington State, King County is a cosmopolitan and thriving economic area. The most populated county in the state, it features an enthralling fusion of suburban, urban, and rural environments, stretching from the Puget Sound shoreline to the spectacular Cascade Mountains. King County, fueled by digital behemoths like Microsoft and Amazon, is a national technological hub that supports a wide range of sectors, including healthcare and aerospace. This vibrant county is a major contributor to the economic and cultural fabric of the Pacific Northwest because of its thriving arts and culture scene, affiliation with esteemed universities such as the University of Washington, and dedication to equity and diversity.

## Business Understanding
In the dynamic real estate market of King County, Washington, numerous households aspire to purchase homes. However, the ever-present information asymmetry often leaves these potential buyers navigating the market blindly. To address this prevalent challenge, our project undertakes an in-depth analysis of house sales data spanning the years 2014 to 2015 within King County. Our mission is to offer invaluable consultation services to a reputable real estate agency dedicated to assisting households in their pursuit of homeownership.

Through a comprehensive examination of this dataset, we aim to bridge the information gap in the real estate market. Our objective is to provide a robust method for predicting house prices, enabling prospective buyers to make well-informed decisions about their property investments. In doing so, we empower both homebuyers and the real estate agency with the knowledge and insights needed to navigate the competitive King County housing landscape effectively.

## Data Understanding

In this section, the identifcation, collection, and surface-level analysis of the data shall be done by:
- Collecting initial data (Has been compiled into a csv file).
- Describing the data we are working with.
- Exploring the data for any relationships and trends.
- Verifying the data quality.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from scipy import stats

%matplotlib inline

In [2]:
raw_df = pd.read_csv('data/kc_house_data.csv')
raw_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,...,8 Good,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


By running the `.info()` method, we can get a closer look at the data in the columns, including the data types and number of null values.

In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

We will further run the `.describe()` method to generate descriptive statistics of the columns.

In [4]:
raw_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


To better understand the dataframe, we will establish its shape as well.

In [5]:
raw_df.shape

(21597, 21)

However, when finding out more information about the dataframe, it was noted that not all columns have the same non-null values. Thus, we will establish exactly how many null values are in each column.

In [6]:
raw_df.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

We will also take the time to establish whether there are any duplicate values in the dataframe. As there are multiple columns with numerical data, duplicates in those columns can be expected. However, when looking at unique columns e.g. the id, duplicates should not be found.

In [7]:
raw_df[raw_df['id'].duplicated(keep=False)].sort_values('id')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2495,1000102,4/22/2015,300000.0,6,3.00,2400,9373,2.0,NO,NONE,...,7 Average,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
2494,1000102,9/16/2014,280000.0,6,3.00,2400,9373,2.0,NaN,NONE,...,7 Average,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
16800,7200179,10/16/2014,150000.0,2,1.00,840,12750,1.0,NO,NONE,...,6 Low Average,840,0.0,1925,0.0,98055,47.4840,-122.211,1480,6969
16801,7200179,4/24/2015,175000.0,2,1.00,840,12750,1.0,NO,NONE,...,6 Low Average,840,0.0,1925,NaN,98055,47.4840,-122.211,1480,6969
11422,109200390,10/20/2014,250000.0,3,1.75,1480,3900,1.0,NO,NONE,...,7 Average,1480,0.0,1980,0.0,98023,47.2977,-122.367,1830,6956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,9828200460,6/27/2014,260000.0,2,1.00,700,4800,1.0,NO,NONE,...,7 Average,700,0.0,1922,0.0,98122,47.6147,-122.300,1440,4800
15186,9834200305,2/10/2015,615000.0,3,1.00,1790,3876,1.5,NO,NONE,...,7 Average,1090,700.0,1904,0.0,98144,47.5750,-122.288,1360,4080
15185,9834200305,7/16/2014,350000.0,3,1.00,1790,3876,1.5,NO,NONE,...,7 Average,1090,?,1904,0.0,98144,47.5750,-122.288,1360,4080
1084,9834200885,7/17/2014,360000.0,4,2.50,2080,4080,1.0,NO,NONE,...,7 Average,1040,1040.0,1962,0.0,98144,47.5720,-122.290,1340,4080


## Data Preparation

The preparation of the final dataset is done by:
- Removal of erroneous data.
- Removal of duplicate data, and 
- Removal of null data.


The process is initiated by dropping certain columns that are deemed unnecessary. These columns are: `date, view, lat, long, sqft_living15` and `sqft_loft15`. This is not to say that other columns may not be dropped later, but this step simply eliminates all columns that are deemed surplus to requirements.

In [8]:
df_cleaned = raw_df.drop(['date', 'view', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],axis=1)
df_cleaned

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
0,7129300520,221900.0,3,1.00,1180,5650,1.0,NaN,Average,7 Average,1180,0.0,1955,0.0
1,6414100192,538000.0,3,2.25,2570,7242,2.0,NO,Average,7 Average,2170,400.0,1951,1991.0
2,5631500400,180000.0,2,1.00,770,10000,1.0,NO,Average,6 Low Average,770,0.0,1933,NaN
3,2487200875,604000.0,4,3.00,1960,5000,1.0,NO,Very Good,7 Average,1050,910.0,1965,0.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,NO,Average,8 Good,1680,0.0,1987,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,360000.0,3,2.50,1530,1131,3.0,NO,Average,8 Good,1530,0.0,2009,0.0
21593,6600060120,400000.0,4,2.50,2310,5813,2.0,NO,Average,8 Good,2310,0.0,2014,0.0
21594,1523300141,402101.0,2,0.75,1020,1350,2.0,NO,Average,7 Average,1020,0.0,2009,0.0
21595,291310100,400000.0,3,2.50,1600,2388,2.0,NaN,Average,8 Good,1600,0.0,2004,0.0


We will proceed to drop the duplicate values that were found in the data understanding section, keeping the first value of the duplicate. It is assumed that duplicate IDs are erroneous.

In [9]:
df_cleaned.drop_duplicates(subset='id',inplace=True,keep='first')
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21420 entries, 0 to 21596
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21420 non-null  int64  
 1   price          21420 non-null  float64
 2   bedrooms       21420 non-null  int64  
 3   bathrooms      21420 non-null  float64
 4   sqft_living    21420 non-null  int64  
 5   sqft_lot       21420 non-null  int64  
 6   floors         21420 non-null  float64
 7   waterfront     19067 non-null  object 
 8   condition      21420 non-null  object 
 9   grade          21420 non-null  object 
 10  sqft_above     21420 non-null  int64  
 11  sqft_basement  21420 non-null  object 
 12  yr_built       21420 non-null  int64  
 13  yr_renovated   17616 non-null  float64
dtypes: float64(4), int64(6), object(4)
memory usage: 2.5+ MB


Ensuring that there are no duplicates

In [10]:
df_cleaned.duplicated().sum()

0

When delving deeper into the results of `df_cleaned.info()`, it is noted that there are columns that contain null values. Thus, any record with a null value in any column shall be dropped.

In [11]:
df_cleaned.dropna(inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15691 entries, 1 to 21596
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15691 non-null  int64  
 1   price          15691 non-null  float64
 2   bedrooms       15691 non-null  int64  
 3   bathrooms      15691 non-null  float64
 4   sqft_living    15691 non-null  int64  
 5   sqft_lot       15691 non-null  int64  
 6   floors         15691 non-null  float64
 7   waterfront     15691 non-null  object 
 8   condition      15691 non-null  object 
 9   grade          15691 non-null  object 
 10  sqft_above     15691 non-null  int64  
 11  sqft_basement  15691 non-null  object 
 12  yr_built       15691 non-null  int64  
 13  yr_renovated   15691 non-null  float64
dtypes: float64(4), int64(6), object(4)
memory usage: 1.8+ MB


Now that the dataframe does not contain any null values, it needs to undergo a bit of formatting.

Starting with the data in the `condition` column. The values in this column are all strings, but through one-hot encoding, we will convert these values to equivalent numerical values. The conversion is outlined in the table below:

| Old String Value | New Numerical Value |
| ---------------- | ------------------- |
| Poor             | 1                   |
| Fair             | 2                   |
| Average          | 3                   |
| Good             | 4                   |
| Very Good        | 5                   |

The same ideology will be applied in the `waterfront` column. As this column has 2 values, either *YES* or *NO*, they will be converted following the table below:

| Old String Value | New Numerical Value |
| ---------------- | ------------------- |
| NO               | 0                   |
| YES              | 1                   |

In [12]:
df_cleaned.loc[:,'condition'] = df_cleaned['condition'].map({'Poor':1, 'Fair':2, 'Average':3, 'Good':4, 'Very Good':5})

df_cleaned.loc[:,'waterfront'] = df_cleaned['waterfront'].map({'NO':0,'YES':1})

df_cleaned

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,3,7 Average,2170,400.0,1951,1991.0
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,5,7 Average,1050,910.0,1965,0.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,3,8 Good,1680,0.0,1987,0.0
5,7237550310,1230000.0,4,4.50,5420,101930,1.0,0,3,11 Excellent,3890,1530.0,2001,0.0
6,1321400060,257500.0,3,2.25,1715,6819,2.0,0,3,7 Average,1715,?,1995,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21591,2997800021,475000.0,3,2.50,1310,1294,2.0,0,3,8 Good,1180,130.0,2008,0.0
21592,263000018,360000.0,3,2.50,1530,1131,3.0,0,3,8 Good,1530,0.0,2009,0.0
21593,6600060120,400000.0,4,2.50,2310,5813,2.0,0,3,8 Good,2310,0.0,2014,0.0
21594,1523300141,402101.0,2,0.75,1020,1350,2.0,0,3,7 Average,1020,0.0,2009,0.0


Furthermore, when looking at the `grade` column, the rating as well as a string description is given in the column. Both these values are unnecessary as they are duplicates, thus only the numerical rating will be kept.

In [13]:
df_cleaned.loc[:,'grade'] = df_cleaned['grade'].str.split(' ').str[0]
df_cleaned

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,3,7,2170,400.0,1951,1991.0
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,5,7,1050,910.0,1965,0.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,3,8,1680,0.0,1987,0.0
5,7237550310,1230000.0,4,4.50,5420,101930,1.0,0,3,11,3890,1530.0,2001,0.0
6,1321400060,257500.0,3,2.25,1715,6819,2.0,0,3,7,1715,?,1995,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21591,2997800021,475000.0,3,2.50,1310,1294,2.0,0,3,8,1180,130.0,2008,0.0
21592,263000018,360000.0,3,2.50,1530,1131,3.0,0,3,8,1530,0.0,2009,0.0
21593,6600060120,400000.0,4,2.50,2310,5813,2.0,0,3,8,2310,0.0,2014,0.0
21594,1523300141,402101.0,2,0.75,1020,1350,2.0,0,3,7,1020,0.0,2009,0.0


However, when looking closely at the `sqft_basement` column, we notice that it has a placeholder value (?), and there are `335` of these question marks. 

In [14]:
df_cleaned['sqft_basement'][df_cleaned['sqft_basement'] == '?'].count()

334

We shall simply drop any records with the placeholder question mark, as we do not know what it could signify and replacing the value may distort the data.

In [15]:
df_cleaned = df_cleaned[df_cleaned['sqft_basement'] != '?']
df_cleaned

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,3,7,2170,400.0,1951,1991.0
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,5,7,1050,910.0,1965,0.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,3,8,1680,0.0,1987,0.0
5,7237550310,1230000.0,4,4.50,5420,101930,1.0,0,3,11,3890,1530.0,2001,0.0
7,2008000270,291850.0,3,1.50,1060,9711,1.0,0,3,7,1060,0.0,1963,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21591,2997800021,475000.0,3,2.50,1310,1294,2.0,0,3,8,1180,130.0,2008,0.0
21592,263000018,360000.0,3,2.50,1530,1131,3.0,0,3,8,1530,0.0,2009,0.0
21593,6600060120,400000.0,4,2.50,2310,5813,2.0,0,3,8,2310,0.0,2014,0.0
21594,1523300141,402101.0,2,0.75,1020,1350,2.0,0,3,7,1020,0.0,2009,0.0


Counter checking whether the placeholder values are still present:

In [16]:
df_cleaned['sqft_basement'][df_cleaned['sqft_basement'] == '?'].count()

0

Lastly, we shall ensure that the newly converted columns have appropriate data types. We will first make a copy of `df_cleaned` to ensure there are no warnings, then change the data types on the copy we have made, aptly named `df`.

In [17]:
df = df_cleaned.copy()

df['grade'] = df_cleaned['grade'].astype(np.int64)
df['waterfront'] = df_cleaned['waterfront'].astype(np.int64)
df['condition'] = df_cleaned['condition'].astype(np.int64)
df['sqft_basement'] = df_cleaned['sqft_basement'].astype(np.float64)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15357 entries, 1 to 21596
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15357 non-null  int64  
 1   price          15357 non-null  float64
 2   bedrooms       15357 non-null  int64  
 3   bathrooms      15357 non-null  float64
 4   sqft_living    15357 non-null  int64  
 5   sqft_lot       15357 non-null  int64  
 6   floors         15357 non-null  float64
 7   waterfront     15357 non-null  int64  
 8   condition      15357 non-null  int64  
 9   grade          15357 non-null  int64  
 10  sqft_above     15357 non-null  int64  
 11  sqft_basement  15357 non-null  float64
 12  yr_built       15357 non-null  int64  
 13  yr_renovated   15357 non-null  float64
dtypes: float64(5), int64(9)
memory usage: 1.8 MB
